In [1]:
import os
import random
import cv2
import numpy as np
import tensorflow as tf

import pathlib

In [2]:
# Directory containing the "armFlapping" videos
# video_dir = r'C:\Users\arjun\Desktop\College\project\SSBD data\armFlapping'
video_dir = r"C:\Users\arjun\Desktop\College\project\SSBD data\ssbd2\armFlapping\armFlapping"

In [3]:
# COMMENT_SR: this cell can be removed
# !kaggle datasets download -d shradheypathak/ssbd3
# !unzip ssbd3.zip
# !pwd
# !cd ssbd2/armFlapping/armFlapping && idx=1 && IFS=$'\n'; for i in `ls`; do mv $i $idx.avi; idx=$(($idx + 1)); done
# !ls ssbd2/armFlapping/armFlapping
# video_dir = r'/content/ssbd2/armFlapping/armFlapping'

# !apt install mediainfo
# !mediainfo video_file.mp4 /content/ssbd2/armFlapping/armFlapping/1.avi
# !ffmpeg -codecs | grep -i avc

# video_path = str(video_f).replace(".avi", ".mp4")
# os.popen("ffmpeg -i '{input}' -c:v copy -c:a copy -f mp4 '{output}'".format(input = video_f, output = video_path))

In [4]:
# Specify the number of classes and videos per class
NUM_CLASSES = 2  # Only one class for armFlapping
VIDEOS_PER_CLASS = 56  # Total number of videos

In [5]:
# !pip install scikit-video
# import skvideo.io
import os

def frames_from_video_directory(video_dir, n_frames, output_size=(224, 224), frame_step=15):
    """
    Creates frames from each video file in the specified directory.

    Args:
      video_dir: Directory containing video files.
      n_frames: Number of frames to be created per video file.
      output_size: Pixel size of the output frame image.
      frame_step: Number of frames to skip before selecting the next frame.

    Return:
      A NumPy array of frames in the shape of (n_frames, height, width, channels).

      # COMMENT_SR:
      - Either the model or this function required changes for it to work.
      - For now, have modified this function to output result in the shape of
        (n_videos, n_frames, height, width, channels), since that seemed more appropriate to me.
      - In case this needs to be in the original format, the model has to be changed accordingly.
    """
    result = []

    # List all video files in the directory
    video_files = [os.path.join(video_dir, f'{i}.mp4') for i in range(1, VIDEOS_PER_CLASS + 1)]

    for video_path in video_files:
        # COMMENT_SR: added this for the extra dimension (refer to the docstring)
        video_frames = []

        # Open the video file
        src = cv2.VideoCapture(video_path)

        if not src.isOpened():
            print(f"Warning: Unable to open video file: {video_path}")
            continue  # Skip to the next video if unable to open

        video_length = int(src.get(cv2.CAP_PROP_FRAME_COUNT))

        need_length = 1 + (n_frames - 1) * frame_step

        if need_length > video_length:
            start = 0
        else:
            max_start = video_length - need_length
            start = random.randint(0, max_start + 1)

        src.set(cv2.CAP_PROP_POS_FRAMES, start)
        ret, frame = src.read()

        if not ret:
            print(f"Unable to read frame for {video_path}")
            continue  
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        video_frames.append(format_frames(frame, output_size))

        for _ in range(n_frames - 1):
            for _ in range(frame_step):
                ret, frame = src.read()
            ret, frame = src.read()
            if not ret:
                print(f"Unable to read frame for {video_path}")
                frame = np.zeros_like(result[0])
#                 continue
            else:
#                 print(f"{video_path}")
                pass
            
            try:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                frame = format_frames(frame, output_size)
                video_frames.append(frame)
            except:
                pass

        src.release()
        result.append(video_frames)

    result = np.array(result)

    return result

In [6]:
def format_frames(frame, output_size):
    """
    Pad and resize an image from a video.

    Args:
      frame: Image that needs to be resized and padded.
      output_size: Pixel size of the output frame image.

    Return:
      Formatted frame with padding of the specified output size.
    """
    frame = tf.image.convert_image_dtype(frame, tf.float32)
    frame = tf.image.resize_with_pad(frame, *output_size)
    return frame

In [7]:
# from google.colab.patches import cv2_imshow
# Function to display frames
def display_frames(frames):
    for video in frames:
        for frame in video:
            # cv2.imshow("Video Frame", frame)
            # cv2_imshow(frame)
            # if cv2.waitKey(1) & 0xFF == ord('q'):
            #     break

            # COMMENT_SR: using matplotlib instead of opencv
            # because of some known codec issues
            from matplotlib import pyplot as plt
            plt.figure()
            plt.imshow(frame)

In [8]:
# dire = r"C:\Users\arjun\Desktop\College\project\SSBD data\ssbd2\armFlapping\armFlapping"
# import os
# for i,f in enumerate(os.listdir(dire)):
#     print(f)
#     f1=os.fsdecode(f)
#     os.rename("%s\%s"%(dire,f1), "%s\%s.mp4"%(dire,i+1))

In [9]:
# Define the number of frames to extract per video
n_frames_per_video = 3  # Set this to your desired number of frames

# Extract frames and display
frames = frames_from_video_directory(video_dir, n_frames_per_video)
# display_frames(frames)

# Close the OpenCV window after displaying frames
# cv2.destroyAllWindows()

Unable to read frame for C:\Users\arjun\Desktop\College\project\SSBD data\ssbd2\armFlapping\armFlapping\42.mp4


In [10]:
# Check the shape of the frames array
if frames is not None:
    print("Shape of frames array:", frames.shape)

Shape of frames array: (56, 3, 224, 224, 3)


In [11]:
import tensorflow as tf

# Define your MobileNetV2-based model
def create_mobilenetv2_model(input_shape):
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=input_shape, include_top=False, weights='imagenet'
    )
    for layer in base_model.layers:
        layer.trainable = False  # Freeze the MobileNetV2 layers

    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    return tf.keras.models.Model(inputs=base_model.input, outputs=x)

# Define a model combining MobileNetV2 and LSTM
def create_mobilenetv2_lstm_model(input_shape, n_frames, lstm_units):
    mobilenetv2_model = create_mobilenetv2_model(input_shape)

    # COMMENT_SR: this layer needs to change in case the original dimension
    # has to be preserved (refer to docstring of frames_from_video_directory)
    frame_input = tf.keras.layers.Input(shape=(n_frames, *input_shape))

    frames = tf.keras.layers.TimeDistributed(mobilenetv2_model)(frame_input)

    lstm_output = tf.keras.layers.LSTM(lstm_units)(frames)

    output = tf.keras.layers.Dense(1, activation='sigmoid')(lstm_output)

    model = tf.keras.models.Model(inputs=frame_input, outputs=output)
    return model

# Define the input shape based on the output_size from your frame extraction code
input_shape = (224, 224, 3)

# Number of LSTM units
lstm_units = 64

# Create the model
model = create_mobilenetv2_lstm_model(input_shape, n_frames_per_video, lstm_units)

# Compile the model with appropriate loss and optimizer
model.compile(
    loss='binary_crossentropy',  # Use appropriate loss function
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),  # Adjust the learning rate
    metrics=['accuracy']  # Add more metrics if needed
)

# Print the model summary
model.summary()

# You can now train the model using your video frames data
# Make sure to have your data and labels ready for training
# model.fit(...)


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 3, 224, 224, 3)   0         
                             ]                                   
                                                                 
 time_distributed (TimeDist  (None, 3, 1280)           2257984   
 ributed)                                                        
                                                                 
 lstm (LSTM)                 (None, 64)                344320    
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 2602369 (9.93 MB)
Trainable params: 344385 (1.31 MB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [12]:
import numpy as np

# Load your video frames and labels
X = frames  # Replace 'frames' with your actual video frames
y = np.array([0] * VIDEOS_PER_CLASS)  # Replace with your actual labels

# Normalize the frame data (you may need to adjust the normalization method)
X = X / 255.0

In [13]:
frames.shape

(56, 3, 224, 224, 3)

In [14]:
# Split your data into training and testing sets
from sklearn.model_selection import train_test_split

# Define your video labels, where 0 represents one class and 1 represents another class
labels = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 , 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  # Replace '...' with labels for each video

# COMMENT_SR: perhaps better to have it this way
labels = [0 for _ in range(48)] + [1 for _ in range(8)]

from sklearn.utils import shuffle
frames_s, labels_s = shuffle(frames, labels)

# Split the frames and labels (assuming you have labels for each video)
X_train, X_test, y_train, y_test = train_test_split(frames_s, labels_s, test_size=0.2, random_state=42)

# COMMENT_SR: since model.fit requires numpy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)

# Train your model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

Epoch 1/10
2/2 [==============================] - 10s 4s/step - loss: nan - accuracy: 0.5909 - val_loss: nan - val_accuracy: 0.9167
Epoch 2/10
2/2 [==============================] - 5s 2s/step - loss: nan - accuracy: 0.8409 - val_loss: nan - val_accuracy: 0.9167
Epoch 3/10
2/2 [==============================] - 5s 2s/step - loss: nan - accuracy: 0.8409 - val_loss: nan - val_accuracy: 0.9167
Epoch 4/10
2/2 [==============================] - 5s 2s/step - loss: nan - accuracy: 0.8409 - val_loss: nan - val_accuracy: 0.9167
Epoch 5/10
2/2 [==============================] - 5s 2s/step - loss: nan - accuracy: 0.8409 - val_loss: nan - val_accuracy: 0.9167
Epoch 6/10
2/2 [==============================] - 5s 2s/step - loss: nan - accuracy: 0.8409 - val_loss: nan - val_accuracy: 0.9167
Epoch 7/10
2/2 [==============================] - 5s 2s/step - loss: nan - accuracy: 0.8409 - val_loss: nan - val_accuracy: 0.9167
Epoch 8/10
2/2 [==============================] - 5s 2s/step - loss: nan - accurac

In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

# Predict on test data
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)  # Convert probabilities to binary predictions

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred_binary)

# Print the metrics
print("Accuracy:", accuracy)

conf_matrix = confusion_matrix(y_test, y_pred_binary)
print("Confusion Matrix:\n", conf_matrix)

classification_rep = classification_report(y_test, y_pred_binary)
print("Classification Report:\n", classification_rep)

1/1 [==============================] - 1s 671ms/step
Accuracy: 0.9166666666666666
Confusion Matrix:
 [[11  0]
 [ 1  0]]
Classification Report:
               precision    recall  f1-score   support

           0       0.92      1.00      0.96        11
           1       0.00      0.00      0.00         1

    accuracy                           0.92        12
   macro avg       0.46      0.50      0.48        12
weighted avg       0.84      0.92      0.88        12



D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
